Create interactive [leaflet](http://leafletjs.com/) map with the hotels and the interesting locations.
# Import 

In [1]:
import folium
import json
from geoip import geolite2
import pycountry
import re
from collections import Counter 
from geopy.geocoders import Nominatim

* [Folium](https://pypi.python.org/pypi/folium) to visualise with Leaflet,
* [json](https://docs.python.org/2/library/json.html) to read country information,
* [pandas](http://pandas.pydata.org/) to read the abstract data,
* [geopy](https://pypi.python.org/pypi/geopy) to get coordinates from location,
* [geopip](https://pypi.python.org/pypi/GeoIP/) to get the location from the IP address,
* [pycountry](https://pypi.python.org/pypi/pycountry) to change the country name into country code.

# Coordinates

In [4]:
geolocator = Nominatim()
location = geolocator.geocode("Place du XX Aout, Liege, Belgium")

Read list of hotels and generate coordinates

In [5]:
hoteldictionnary = {"IBIS CENTRE OPERA ***" : 'pl. de la République française, 41 - 4000 Liège',
                    "BEST WESTERN UNIVERS ***" : 'rue des Guillemins, 116 - 4000 Liège',
                    "HUSA COURONNE ***" : 'Place des Guillemins, 11 - 4000 Liège',
                    "PentaHotel ****" : 'bd de la Sauvenière, 100 - 4000 Liège',
                    "CROWNE PLAZA *****" : 'Mont Saint-Martin 9-11 - 4000 Liège',
                    "CYGNE D'ARGENT **": 'rue Beeckman, 49 - 4000 Liège',
                    "Alliance Hotel ****" : "Palais des Congrès esplanade de l'Europe, 2B - 4020 Liège",
                    "HORS CHATEAU ***" : 'Impasse des Drapiers, 2 - 4000 Liège',
                    "B&B THE STREET LODGE" : 'Rue Dartois 17A 4000 Liège',
                    "HOTEL RIAD ***" : 'rue des Vennes 166 B-4020 Liège',
                    "HOTEL NEUVICE ***" : 'En Neuvice 45 B-4000 Liège',
                    "La PASSERELLE ***" : 'chaussée des Prés, 24 - 4020 Liège (Outremeuse)',
                    "EUROTEL **" : 'rue Léon Frédéricq, 29 - 4020 Liège',
                    "AUBERGE DE JEUNESSE GEORGES SIMENON (Youth hostel)" : 'Rue Georges Simenon, 2 - 4020 Liège',
                    "SAINT-GANGULPHE - ApartHotel" : 'Rue Saint-Gangulphe 21, 4000 Liège',
                    "Ramada Plazza ***" : 'Quai Saint-Léonard, 36 - 4000 Liège',
                    "LES ACTEURS **" : 'rue des Urbanistes, 10 - 4000 Liège'
                    }

In [8]:
hoteldictionnary2 = {}
for hotelnames, hoteladdress in hoteldictionnary.items():
    hotelgeoloc = geolocator.geocode(hoteladdress)
    if hotelgeoloc is not None:
        hoteldictionnary2[hotelnames] = (hotelgeoloc.latitude, hotelgeoloc.longitude)

In [9]:
hoteldictionnary2

{'AUBERGE DE JEUNESSE GEORGES SIMENON (Youth hostel)': (50.641546, 5.5843609),
 'BEST WESTERN UNIVERS ***': (50.6300641, 5.5685016),
 'CROWNE PLAZA *****': (50.64479745, 5.5676806088668),
 "CYGNE D'ARGENT **": (50.63502765, 5.56554641274591),
 'EUROTEL **': (50.63341035, 5.57572037713675),
 'HOTEL NEUVICE ***': (50.6449849, 5.5769297),
 'HOTEL RIAD ***': (50.6224405, 5.5907478),
 'HUSA COURONNE ***': (50.6257666, 5.56652974753835),
 'LES ACTEURS **': (50.64112505, 5.56614356041927),
 'La PASSERELLE ***': (50.6420617, 5.58049993902227),
 'PentaHotel ****': (50.6437184, 5.5664008),
 'Ramada Plazza ***': (50.6479396, 5.5905565)}

In [10]:
hoteldictionnary2 = {
    'AUBERGE DE JEUNESSE GEORGES SIMENON (Youth hostel)': (50.641546, 5.5843609),
    'Alliance Hotel ****': (50.631269, 5.574348),
    'B&B THE STREET LODGE': (50.6285155, 5.56639087398867),
    'BEST WESTERN UNIVERS ***': (50.6287652, 5.5677144),
    'CROWNE PLAZA *****': (50.6451365, 5.5678528),
    "LE CYGNE D'ARGENT **": (50.63502765, 5.56554641274591),
    'EUROTEL **': (50.6334275, 5.5756941),
    'HOTEL NEUVICE ***': (50.6449849, 5.5769297),
    'HOTEL RIAD ***': (50.6252358, 5.5873542),
    'HUSA COURONNE ***': (50.6262267, 5.5658734),
    'IBIS CENTRE OPERA ***': (50.6430835, 5.5725104),
    'La PASSERELLE ***': (50.6420617, 5.58049993902227),
    'Penta Hotel ****': (50.6429273, 5.5654273),
    'Ramada Plaza ***': (50.6479396, 5.5905565)}

In [11]:
hoteldictionnary2['Les Acteurs'] = (50.641068, 5.566224)
hoteldictionnary2['SAINT-GANGULPHE - ApartHotel'] = (50.643651, 5.573566)
hoteldictionnary2['HORS CHATEAU'] = (50.647197, 5.579228)
hoteldictionnary2['HOTEL RIAD ***'] = (50.625717, 5.586460)
hoteldictionnary2['BEST WESTERN UNIVERS ***'] = (50.626396, 5.565975)
hoteldictionnary2['HUSA COURONNE\n***'] = (50.625790, 5.566651)

# Create the map

We initialize the map with the center on Belgium and with a global view (zoom = 1.5).

Icons taken from http://getbootstrap.com/components/

In [15]:
map_clq = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=14)
marker = folium.Marker([50.640960, 5.575476], popup='Colloquium Venue',
                      icon=folium.Icon(color='red',icon='home'))
map_clq.add_child(marker)

for hotelname, hotelcoord in hoteldictionnary2.items():
    marker = folium.Marker([hotelcoord[0], hotelcoord[1]], popup=hotelname)
    map_clq.add_child(marker)
map_clq    

In [65]:
map_clq.save('./map_hotels_clq.html')